### Import Neccesary Libraries

In [1]:
import pandas as pd
import re

### Fetch, inspect, transform and clean required datasets

####  Accounts dataset 

##### Prep

In [2]:
account_df = pd.read_csv('../datasets/accounts.csv')
account_df.head()

,account_id,signup_date,utm_source,utm_campaign,ref,account_type
0,ACC1000,2025-07-29,NaN,Campaign_C,ability,agency
1,ACC1001,2025-06-13,NaN,Campaign_A,NaN,AGENCY
2,ACC1002,2025-06-03,LinkedIn,Campaign_B,NaN,normal
3,ACC1003,2025/08/01,NaN,Campaing_B,start,AGENCY
4,ACC1004,2025-07-24,LinkedIn,Campaing_B,NaN,Normal


In [3]:
account_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5010 entries, 0 to 5009
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   account_id    5010 non-null   object
 1   signup_date   5010 non-null   object
 2   utm_source    4217 non-null   object
 3   utm_campaign  5010 non-null   object
 4   ref           1720 non-null   object
 5   account_type  5010 non-null   object
dtypes: object(6)
memory usage: 235.0+ KB


In [4]:
account_df.describe()

,account_id,signup_date,utm_source,utm_campaign,ref,account_type
count,5010,5010,4217,5010,1720,5010
unique,5000,93,5,4,801,4
top,ACC1000,2025/08/01,Organic,Campaing_B,follow,AGENCY
freq,2,1732,860,1271,8,1297


In [5]:
print(account_df['utm_source'].unique())
print(account_df['utm_campaign'].unique())
print(account_df['account_type'].unique())

[nan 'LinkedIn' 'Google Ads' 'Googel Ads' 'Organic' 'Referral']
['Campaign_C' 'Campaign_A' 'Campaign_B' 'Campaing_B']
['agency' 'AGENCY' 'normal' 'Normal']


In [6]:
# Check for duplicate account_id records as we see in .describe() 5000 unqiue account ids and 5010 records
duplicates = account_df[account_df.duplicated(keep=False)]
duplicates

,account_id,signup_date,utm_source,utm_campaign,ref,account_type
0,ACC1000,2025-07-29,NaN,Campaign_C,ability,agency
1,ACC1001,2025-06-13,NaN,Campaign_A,NaN,AGENCY
2,ACC1002,2025-06-03,LinkedIn,Campaign_B,NaN,normal
3,ACC1003,2025/08/01,NaN,Campaing_B,start,AGENCY
4,ACC1004,2025-07-24,LinkedIn,Campaing_B,NaN,Normal
5,ACC1005,07/15/2025,Google Ads,Campaign_A,NaN,normal
6,ACC1006,07/15/2025,Googel Ads,Campaign_C,marriage,AGENCY
7,ACC1007,07/15/2025,Google Ads,Campaign_C,NaN,Normal
8,ACC1008,2025/08/01,Google Ads,Campaign_A,NaN,agency
9,ACC1009,2025-08-10,LinkedIn,Campaign_A,NaN,Normal


In [15]:
# Drop the duplicate account id records as visible above
account_df = account_df.drop_duplicates()

# Convert 'signup_date' to datetime standard format
account_df['signup_date'] = pd.to_datetime(account_df['signup_date'], format='mixed',errors='coerce')

# Standardize account_type values
account_df['account_type'] = account_df['account_type'].str.title().str.strip()

# Cleanup utm_source values
account_df['utm_source'] = account_df['utm_source'].replace('Googel Ads','Google Ads')
account_df['utm_source'] = account_df['utm_source'].fillna('Unknown')

# Cleanup utm_campaign values
account_df['utm_campaign'] = account_df['utm_campaign'].replace('Campaing_B', 'Campaign_B')

# add month_year column
account_df['month_year'] = account_df['signup_date'].dt.to_period('M')

account_df.head(10)

,account_id,signup_date,utm_source,utm_campaign,ref,account_type,month_year
0,ACC1000,2025-07-29,Unknown,Campaign_C,ability,Agency,2025-07
1,ACC1001,2025-06-13,Unknown,Campaign_A,NaN,Agency,2025-06
2,ACC1002,2025-06-03,LinkedIn,Campaign_B,NaN,Normal,2025-06
3,ACC1003,2025-08-01,Unknown,Campaign_B,start,Agency,2025-08
4,ACC1004,2025-07-24,LinkedIn,Campaign_B,NaN,Normal,2025-07
5,ACC1005,2025-07-15,Google Ads,Campaign_A,NaN,Normal,2025-07
6,ACC1006,2025-07-15,Google Ads,Campaign_C,marriage,Agency,2025-07
7,ACC1007,2025-07-15,Google Ads,Campaign_C,NaN,Normal,2025-07
8,ACC1008,2025-08-01,Google Ads,Campaign_A,NaN,Agency,2025-08
9,ACC1009,2025-08-10,LinkedIn,Campaign_A,NaN,Normal,2025-08


In [16]:
account_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   account_id    5000 non-null   object        
 1   signup_date   5000 non-null   datetime64[ns]
 2   utm_source    5000 non-null   object        
 3   utm_campaign  5000 non-null   object        
 4   ref           1717 non-null   object        
 5   account_type  5000 non-null   object        
 6   month_year    5000 non-null   period[M]     
dtypes: datetime64[ns](1), object(5), period[M](1)
memory usage: 312.5+ KB


In [17]:
account_df.describe(include='all')

,account_id,signup_date,utm_source,utm_campaign,ref,account_type,month_year
count,5000,5000,5000,5000,1717,5000,5000
unique,5000,NaN,5,3,801,2,3
top,ACC1000,NaN,Google Ads,Campaign_B,follow,Agency,2025-08
freq,1,NaN,1675,2534,8,2567,2274
mean,NaN,2025-07-21 04:18:02.880000,NaN,NaN,NaN,NaN,NaN
min,NaN,2025-06-01 00:00:00,NaN,NaN,NaN,NaN,NaN
25%,NaN,2025-07-15 00:00:00,NaN,NaN,NaN,NaN,NaN
50%,NaN,2025-07-18 00:00:00,NaN,NaN,NaN,NaN,NaN
75%,NaN,2025-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN
max,NaN,2025-08-30 00:00:00,NaN,NaN,NaN,NaN,NaN


##### Dataset

In [18]:
account_df.sample(20)

,account_id,signup_date,utm_source,utm_campaign,ref,account_type,month_year
4499,ACC5499,2025-06-09,Google Ads,Campaign_B,enough,Agency,2025-06
907,ACC1907,2025-08-01,Google Ads,Campaign_C,follow,Normal,2025-08
758,ACC1758,2025-08-01,Organic,Campaign_A,NaN,Agency,2025-08
3222,ACC4222,2025-07-15,Organic,Campaign_A,yet,Agency,2025-07
3589,ACC4589,2025-07-07,Google Ads,Campaign_A,reveal,Normal,2025-07
4273,ACC5273,2025-07-22,Unknown,Campaign_B,NaN,Normal,2025-07
3107,ACC4107,2025-06-07,Referral,Campaign_C,NaN,Normal,2025-06
3559,ACC4559,2025-07-09,Organic,Campaign_C,NaN,Normal,2025-07
4958,ACC5958,2025-07-27,Google Ads,Campaign_A,let,Normal,2025-07
2515,ACC3515,2025-07-15,Google Ads,Campaign_B,NaN,Agency,2025-07


##### Inspection

In [14]:
# Set signup_date as index for easier time-based operations
account_df_time = account_df.set_index('signup_date')

# Aggregate signups by Month
monthly_signups = account_df_time.resample('M').size()
monthly_signups.name = 'MonthlySignups'
print("\nSignups per Month:")
print(monthly_signups)


Signups per Month:
signup_date
2025-06-30     544
2025-07-31    2182
2025-08-31    2274
Freq: M, Name: MonthlySignups, dtype: int64


In [19]:
# Pivot on month and utm_source
accounts_utm_source_wise = account_df.pivot_table(
    index="month_year",        # rows
    columns="utm_source",     # columns
    values="account_id",      # what to count
    aggfunc="count",     # how to aggregate
    fill_value=0         # replace NaN with 0
)

print(accounts_utm_source_wise)

utm_source  Google Ads  LinkedIn  Organic  Referral  Unknown
month_year                                                  
2025-06            181        91       89        91       92
2025-07            742       376      351       373      340
2025-08            752       381      420       363      358


In [20]:
# Pivot on month and utm_campaign
accounts_utm_campaign_wise = account_df.pivot_table(
    index="month_year",        # rows
    columns="utm_campaign",     # columns
    values="account_id",      # what to count
    aggfunc="count",     # how to aggregate
    fill_value=0         # replace NaN with 0
)

print(accounts_utm_campaign_wise)

utm_campaign  Campaign_A  Campaign_B  Campaign_C
month_year                                      
2025-06              143         262         139
2025-07              536        1122         524
2025-08              578        1150         546


In [21]:
# Pivot on month and utm_campaign
accounts_account_type_wise = account_df.pivot_table(
    index="month_year",        # rows
    columns="account_type",     # columns
    values="account_id",      # what to count
    aggfunc="count",     # how to aggregate
    fill_value=0         # replace NaN with 0
)

print(accounts_account_type_wise)

account_type  Agency  Normal
month_year                  
2025-06          288     256
2025-07         1109    1073
2025-08         1170    1104


In [ ]:
# Analyze campaign combinations
top_campaign_combos = account_df.groupby(['utm_source', 'utm_campaign']).size().nlargest(5)
print("\nTop 5 Performing Source/Campaign Combinations (by volume):")
print(top_campaign_combos)


Top 5 Performing Source/Campaign Combinations (by volume):
utm_source  utm_campaign
Google Ads  Campaign_B      856
Organic     Campaign_B      437
Referral    Campaign_B      429
LinkedIn    Campaign_B      428
Google Ads  Campaign_A      422
dtype: int64


#### ad performance dataset

##### Prep

In [22]:
ad_performance_df = pd.read_json('../datasets/ad_performance.json')
ad_performance_df.head()

,campaign_id,campaign_name,date,impressions,clicks,cost,conversions
0,CAMP101,Campain_C,2025-09-01,-7,0,1216.84,4
1,CAMP101,Campaign_A,2025-07-24,0,5,305.48,1
2,CAMP103,Campaign_C,2025-07-16,256,154,1985.38,52
3,CAMP107,Campaign_A,2025-06-26,0,1,751.43,11
4,CAMP108,Campaign_A,2025-06-27,269,22,289.38,8


In [23]:
ad_performance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   campaign_id    500 non-null    object        
 1   campaign_name  500 non-null    object        
 2   date           500 non-null    datetime64[ns]
 3   impressions    500 non-null    int64         
 4   clicks         500 non-null    int64         
 5   cost           500 non-null    float64       
 6   conversions    500 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 27.5+ KB


In [24]:
ad_performance_df.describe(include='all')

,campaign_id,campaign_name,date,impressions,clicks,cost,conversions
count,500,500,500,500.000000,500.000000,500.000000,500.000000
unique,11,4,NaN,NaN,NaN,NaN,NaN
top,CAMP107,Campain_C,NaN,NaN,NaN,NaN,NaN
freq,56,132,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2025-08-08 14:00:57.600000,259.832000,133.766000,1031.820540,69.940000
min,NaN,NaN,2025-06-01 00:00:00,-7.000000,0.000000,3.890000,0.000000
25%,NaN,NaN,2025-07-16 00:00:00,0.000000,2.000000,511.095000,5.000000
50%,NaN,NaN,2025-08-28 00:00:00,0.000000,5.000000,1060.440000,11.000000
75%,NaN,NaN,2025-09-01 00:00:00,538.500000,215.500000,1519.137500,80.250000
max,NaN,NaN,2025-09-01 00:00:00,998.000000,865.000000,1995.700000,826.000000


In [25]:
print(ad_performance_df['campaign_name'].unique())
print(ad_performance_df['campaign_id'].unique())

['Campain_C' 'Campaign_A' 'Campaign_C' 'Campaign_B']
['CAMP101' 'CAMP103' 'CAMP107' 'CAMP108' 'CAMP109' 'CAMP104' 'CAMP102'
 'CAMP110' 'CAMP105' 'CAMP100' 'CAMP106']


In [27]:
# cleanup 'campaign_name' values
ad_performance_df['campaign_name'] = ad_performance_df['campaign_name'].str.replace('Campain_C', 'Campaign_C')

# add a month_year column
ad_performance_df['month_year'] = pd.to_datetime(ad_performance_df['date']).dt.to_period('M')

In [28]:
ad_performance_df.describe(include='all')

,campaign_id,campaign_name,date,impressions,clicks,cost,conversions,month_year
count,500,500,500,500.000000,500.000000,500.000000,500.000000,500
unique,11,3,NaN,NaN,NaN,NaN,NaN,4
top,CAMP107,Campaign_C,NaN,NaN,NaN,NaN,NaN,2025-09
freq,56,257,NaN,NaN,NaN,NaN,NaN,243
mean,NaN,NaN,2025-08-08 14:00:57.600000,259.832000,133.766000,1031.820540,69.940000,NaN
min,NaN,NaN,2025-06-01 00:00:00,-7.000000,0.000000,3.890000,0.000000,NaN
25%,NaN,NaN,2025-07-16 00:00:00,0.000000,2.000000,511.095000,5.000000,NaN
50%,NaN,NaN,2025-08-28 00:00:00,0.000000,5.000000,1060.440000,11.000000,NaN
75%,NaN,NaN,2025-09-01 00:00:00,538.500000,215.500000,1519.137500,80.250000,NaN
max,NaN,NaN,2025-09-01 00:00:00,998.000000,865.000000,1995.700000,826.000000,NaN


##### Dataset

In [ ]:
ad_performance_df.head(50)

,campaign_id,campaign_name,date,impressions,clicks,cost,conversions
0,CAMP101,Campaign_C,2025-09-01,-7,0,1216.84,4
1,CAMP101,Campaign_A,2025-07-24,0,5,305.48,1
2,CAMP103,Campaign_C,2025-07-16,256,154,1985.38,52
3,CAMP107,Campaign_A,2025-06-26,0,1,751.43,11
4,CAMP108,Campaign_A,2025-06-27,269,22,289.38,8
5,CAMP109,Campaign_C,2025-06-07,0,1,133.95,4
6,CAMP104,Campaign_C,2025-09-01,0,2,373.29,8
7,CAMP102,Campaign_C,2025-08-10,560,507,806.42,380
8,CAMP103,Campaign_C,2025-06-28,399,303,431.29,299
9,CAMP110,Campaign_B,2025-09-01,344,81,1395.40,11


##### Inspection

In [32]:
campaign_wise_clicks = ad_performance_df.pivot_table(
    index="month_year",        # rows
    columns="campaign_name",     # columns
    values="clicks",      # what to count
    aggfunc="sum",     # how to aggregate
    fill_value=0         # replace NaN with 0
)

print(campaign_wise_clicks)

campaign_name  Campaign_A  Campaign_B  Campaign_C
month_year                                       
2025-06              1733        2650        5964
2025-07              2277        1938        7519
2025-08              4011        3347        7591
2025-09              5879        7756       16218


In [ ]:
campaign_wise_clicks = ad_performance_df.pivot_table(
    index="month_year",        # rows
    columns="campaign_name",     # columns
    values="cost",      # what to count
    aggfunc="sum",     # how to aggregate
    fill_value=0         # replace NaN with 0
)

print(campaign_wise_clicks)

In [33]:
campaign_wise_cost = ad_performance_df.pivot_table(
    index="month_year",        # rows
    columns="campaign_name",     # columns
    values="cost",      # what to count
    aggfunc="sum",     # how to aggregate
    fill_value=0         # replace NaN with 0
)

print(campaign_wise_cost)

campaign_name  Campaign_A  Campaign_B  Campaign_C
month_year                                       
2025-06          25935.34    19490.73    40727.63
2025-07          21812.19    16288.95    46637.49
2025-08          24843.57    21730.27    52893.71
2025-09          62054.69    60510.55   122985.15


#### invoices dataset

##### Prep

In [34]:
invoices_df = pd.read_csv('../datasets/invoices.tsv', sep='\t')
invoices_df.head()

,invoice_id,subscription_id,account_id,invoice_date,amount,currency,status
0,INV5000,SUB3890,ACC1638,2025-09-15,99.43,EUR,PAID
1,INV5001,SUB2773,ACC2322,2025-06-25,0.00,EUR,refunded
2,INV5002,SUB3219,ACC3121,2025-09-15,472.47,USD,paid
3,INV5003,SUB3706,ACC2800,2025-08-29,160.58,USD,PAID
4,INV5004,SUB4196,ACC2384,2025-06-25,0.00,USDD,PAID


In [35]:
invoices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1690 entries, 0 to 1689
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   invoice_id       1690 non-null   object 
 1   subscription_id  1690 non-null   object 
 2   account_id       1690 non-null   object 
 3   invoice_date     1690 non-null   object 
 4   amount           1690 non-null   float64
 5   currency         1690 non-null   object 
 6   status           1690 non-null   object 
dtypes: float64(1), object(6)
memory usage: 92.6+ KB


In [36]:
invoices_df.describe(include='all')

,invoice_id,subscription_id,account_id,invoice_date,amount,currency,status
count,1690,1690,1690,1690,1690.000000,1690,1690
unique,1680,1243,1438,92,NaN,4,4
top,INV5000,SUB3847,ACC2816,2025-09-15,NaN,USDD,PAID
freq,2,4,4,844,NaN,451,456
mean,NaN,NaN,NaN,NaN,118.076278,NaN,NaN
std,NaN,NaN,NaN,NaN,160.961345,NaN,NaN
min,NaN,NaN,NaN,NaN,-49.370000,NaN,NaN
25%,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,232.660000,NaN,NaN


In [37]:
print(invoices_df['currency'].unique())
print(invoices_df['status'].unique())

['EUR' 'USD' 'USDD' 'INR']
['PAID' 'refunded' 'paid' 'failed']


In [ ]:
# Drop duplicate invoice_id records as we see in .describe() 1680 unqiue invoice ids and 1690 records
invoices_df = invoices_df.drop_duplicates()

# Convert 'invoice_date' to datetime standard format
invoices_df['invoice_date'] = pd.to_datetime(invoices_df['invoice_date'], format='mixed',errors='coerce')

# Standardize currency values
invoices_df['currency'] = invoices_df['currency'].replace('USDD','USD')

# Standardize status values
invoices_df['status'] = invoices_df['status'].str.lower().str.strip()

# Standardize amount values to INR
exchange_rates = {
    "INR": 1,
    "USD": 83,   # 1 USD = 83 INR
    "EUR": 90    # 1 EUR = 90 INR
}
invoices_df["amount_in_inr"] = invoices_df.apply(lambda row: row["amount"] * exchange_rates[row["currency"]], axis=1)
invoices_df.head()

,invoice_id,subscription_id,account_id,invoice_date,amount,currency,status,amount_in_inr
0,INV5000,SUB3890,ACC1638,2025-09-15,99.43,EUR,paid,8948.70
1,INV5001,SUB2773,ACC2322,2025-06-25,0.00,EUR,refunded,0.00
2,INV5002,SUB3219,ACC3121,2025-09-15,472.47,USD,paid,39215.01
3,INV5003,SUB3706,ACC2800,2025-08-29,160.58,USD,paid,13328.14
4,INV5004,SUB4196,ACC2384,2025-06-25,0.00,USD,paid,0.00


In [160]:
invoices_df['currency'].value_counts()

currency
USD    873
EUR    422
INR    385
Name: count, dtype: int64

In [105]:
invoices_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1680 entries, 0 to 1679
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   invoice_id       1680 non-null   object        
 1   subscription_id  1680 non-null   object        
 2   account_id       1680 non-null   object        
 3   invoice_date     1680 non-null   datetime64[ns]
 4   amount           1680 non-null   float64       
 5   currency         1680 non-null   object        
 6   status           1680 non-null   object        
 7   amount_in_inr    1680 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 118.1+ KB


In [106]:
invoices_df.describe(include='all')

,invoice_id,subscription_id,account_id,invoice_date,amount,currency,status,amount_in_inr
count,1680,1680,1680,1680,1680.000000,1680,1680,1680.000000
unique,1680,1243,1438,NaN,NaN,3,3,NaN
top,INV5000,SUB3847,ACC2816,NaN,NaN,USD,paid,NaN
freq,1,4,4,NaN,NaN,873,873,NaN
mean,NaN,NaN,NaN,2025-08-14 19:30:51.428571392,118.351024,NaN,NaN,7779.323810
min,NaN,NaN,NaN,2025-06-01 00:00:00,-49.370000,NaN,NaN,-4443.300000
25%,NaN,NaN,NaN,2025-07-15 00:00:00,0.000000,NaN,NaN,0.000000
50%,NaN,NaN,NaN,2025-09-15 00:00:00,0.000000,NaN,NaN,0.000000
75%,NaN,NaN,NaN,2025-09-15 00:00:00,233.547500,NaN,NaN,13190.602500
max,NaN,NaN,NaN,2025-09-15 00:00:00,498.520000,NaN,NaN,44800.200000


##### Dataset

In [107]:
invoices_df.sample(10)

,invoice_id,subscription_id,account_id,invoice_date,amount,currency,status,amount_in_inr
717,INV5717,SUB2959,ACC5400,2025-07-20,471.76,INR,failed,471.76
97,INV5097,SUB3508,ACC2166,2025-09-15,487.45,EUR,paid,43870.50
699,INV5699,SUB2803,ACC2893,2025-09-15,3.63,USD,failed,301.29
604,INV5604,SUB4460,ACC2051,2025-09-15,309.06,INR,paid,309.06
574,INV5574,SUB2273,ACC4672,2025-09-15,0.00,USD,paid,0.00
177,INV5177,SUB3396,ACC5101,2025-09-15,56.09,USD,failed,4655.47
818,INV5818,SUB3227,ACC4697,2025-06-19,491.76,EUR,failed,44258.40
1056,INV6056,SUB3025,ACC971,2025-09-15,0.00,INR,failed,0.00
906,INV5906,SUB1905,ACC3266,2025-09-15,227.67,USD,failed,18896.61
249,INV5249,SUB2318,ACC969,2025-08-22,0.00,EUR,paid,0.00


#### Inspect

In [108]:
currency_counts = invoices_df['currency'].value_counts()
currency_counts

currency
USD    873
EUR    422
INR    385
Name: count, dtype: int64

In [112]:
amount_earned_from_different_currencies = invoices_df.groupby('currency')['amount_in_inr'].sum().sort_values(ascending=False)

amount_earned_from_different_currencies

currency
USD    8513094.2
EUR    4510018.8
INR      46151.0
Name: amount_in_inr, dtype: float64

#### server events dataset

##### Prep

In [41]:
with open("../datasets/server_events.log", "r") as f:
    lines = f.readlines()

records = []
for line in lines:
    parts = line.strip().split(maxsplit=1)   # split into date and rest
    if len(parts) < 2:
        continue
    
    date, rest = parts
    fields= {'date': date}
    fields.update(dict(re.findall(r'(\w+)=([\w\/_]*)', rest)))  # capture key=value pairs
    records.append(fields)

# Convert to DataFrame
server_events_df = pd.DataFrame(records)

server_events_df.head()

,date,account_id,event,page,type,feature
0,2025-06-16,,page_view,/features,NaN,NaN
1,07-15-2025,ACC5981,error,NaN,load_error,NaN
2,07-15-2025,ACC2061,feature_use,NaN,NaN,cookie_banner
3,07-15-2025,,page_view,,NaN,NaN
4,07-15-2025,,error,NaN,unknown,NaN


In [42]:
server_events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        10000 non-null  object
 1   account_id  10000 non-null  object
 2   event       10000 non-null  object
 3   page        3275 non-null   object
 4   type        3348 non-null   object
 5   feature     3377 non-null   object
dtypes: object(6)
memory usage: 468.9+ KB


In [43]:
server_events_df.describe()

,date,account_id,event,page,type,feature
count,10000,10000,10000,3275,3348,3377
unique,92,3198,3,4,3,3
top,07-15-2025,,feature_use,,payment_failed,
freq,4982,4971,3377,839,1157,1139


In [44]:
print(server_events_df['event'].unique())
print(server_events_df['page'].unique())
print(server_events_df['type'].unique())
print(server_events_df['feature'].unique())

['page_view' 'error' 'feature_use']
['/features' nan '' '/pricing' '/home']
[nan 'load_error' 'unknown' 'payment_failed']
[nan 'cookie_banner' '' 'policy_generator']


In [49]:
# Convert 'date' to datetime standard format
server_events_df['date'] = pd.to_datetime(server_events_df['date'], format='mixed', errors='coerce')
server_events_df.head()

# extract month_year from date
server_events_df['month_year'] = server_events_df['date'].dt.to_period('M')

In [50]:
server_events_df.describe(include='all')

,date,account_id,event,page,type,feature,month_year
count,10000,10000,10000,3275,3348,3377,10000
unique,NaN,3198,3,4,3,3,3
top,NaN,,feature_use,,payment_failed,,2025-07
freq,NaN,4971,3377,839,1157,1139,6703
mean,2025-07-15 02:44:00.960000,NaN,NaN,NaN,NaN,NaN,NaN
min,2025-06-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
25%,2025-07-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
50%,2025-07-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
75%,2025-07-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
max,2025-08-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
server_events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        10000 non-null  datetime64[ns]
 1   account_id  10000 non-null  object        
 2   event       10000 non-null  object        
 3   page        3275 non-null   object        
 4   type        3348 non-null   object        
 5   feature     3377 non-null   object        
 6   month_year  10000 non-null  period[M]     
dtypes: datetime64[ns](1), object(5), period[M](1)
memory usage: 547.0+ KB


##### Dataset

In [48]:
server_events_df.head(50)

,date,account_id,event,page,type,feature
0,2025-06-16,,page_view,/features,NaN,NaN
1,2025-07-15,ACC5981,error,NaN,load_error,NaN
2,2025-07-15,ACC2061,feature_use,NaN,NaN,cookie_banner
3,2025-07-15,,page_view,,NaN,NaN
4,2025-07-15,,error,NaN,unknown,NaN
5,2025-07-15,,error,NaN,load_error,NaN
6,2025-07-12,ACC3989,error,NaN,load_error,NaN
7,2025-07-15,ACC3902,error,NaN,unknown,NaN
8,2025-08-29,ACC5315,error,NaN,load_error,NaN
9,2025-07-15,,feature_use,NaN,NaN,


#### Inspection

In [54]:
month_wise_events = server_events_df.pivot_table(
    index="month_year",        # rows
    columns="event",     # columns
    values="account_id",      # what to count
    aggfunc="count",     # how to aggregate
    fill_value=0         # replace NaN with 0
)

month_wise_events

event,error,feature_use,page_view
month_year,,,
2025-06,522,595,568
2025-07,2274,2249,2180
2025-08,552,533,527


#### subscriptions dataset

##### Prep

In [127]:
subscrptions_df = pd.read_csv('../datasets/subscriptions.csv')
subscrptions_df.head(10)

,subscription_id,account_id,plan_name,plan_type,billing_cycle_start_date,billing_cycle_end_date,status,is_trial,trial_start_date,trial_end_date,conversion_flag,renewal_count,cancel_reason
0,SUB2000,ACC1211,Enterprise,montly,2025-08-23,2025-08-18,canceled,False,2025-06-03,2025-06-07,no,NaN,competitor
1,SUB2001,ACC4402,Basic,monthly,2025-07-25,2025-08-15,trialing,True,2025-08-02,2025-07-16,no,-1.0,NaN
2,SUB2002,ACC1576,Pro,montly,2025-06-28,2025-07-09,CANCELED,False,2025-07-17,2025-06-26,no,-1.0,NaN
3,SUB2003,ACC5435,Basic,montly,2025-07-26,2025-08-04,Active,False,2025-08-28,2025-06-02,no,NaN,price
4,SUB2004,ACC2856,Enterprise,monthly,2025-06-01,2025-07-23,Active,False,2025-08-22,2025-06-29,yes,NaN,NaN
5,SUB2005,ACC2381,Basic,monthly,2025-06-24,2025-06-07,CANCELED,False,2025-07-05,2025-07-15,yes,1.0,churned
6,SUB2006,ACC3025,Pro,montly,2025-08-07,2025-06-16,canceled,False,2025-08-15,2025-06-18,no,0.0,churned
7,SUB2007,ACC2313,Enterprise,monthly,2025-08-11,2025-07-29,trialing,False,2025-07-08,2025-07-14,yes,0.0,price
8,SUB2008,ACC3952,Basic,montly,2025-08-01,2025-06-13,active,False,2025-08-21,2025-08-10,no,1.0,competitor
9,SUB2009,ACC2324,Enterprise,monthly,2025-07-15,2025-07-02,CANCELED,False,2025-08-04,2025-08-23,no,2.0,churned


In [128]:
subscrptions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2505 entries, 0 to 2504
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   subscription_id           2505 non-null   object 
 1   account_id                2505 non-null   object 
 2   plan_name                 2505 non-null   object 
 3   plan_type                 2505 non-null   object 
 4   billing_cycle_start_date  2505 non-null   object 
 5   billing_cycle_end_date    2505 non-null   object 
 6   status                    2505 non-null   object 
 7   is_trial                  2505 non-null   bool   
 8   trial_start_date          2505 non-null   object 
 9   trial_end_date            2505 non-null   object 
 10  conversion_flag           2505 non-null   object 
 11  renewal_count             1681 non-null   float64
 12  cancel_reason             1888 non-null   object 
dtypes: bool(1), float64(1), object(11)
memory usage: 237.4+ KB


In [129]:
subscrptions_df.describe(include='all')

,subscription_id,account_id,plan_name,plan_type,billing_cycle_start_date,billing_cycle_end_date,status,is_trial,trial_start_date,trial_end_date,conversion_flag,renewal_count,cancel_reason
count,2505,2505,2505,2505,2505,2505,2505,2505,2505,2505,2505,1681.000000,1888
unique,2500,1977,3,3,91,91,5,2,91,91,2,NaN,3
top,SUB2000,ACC2856,Pro,monthly,2025-06-18,2025-07-17,trialing,False,2025-08-22,2025-08-13,no,NaN,churned
freq,2,5,856,850,40,41,522,1317,42,45,1278,NaN,646
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.501487,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.110085,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN


In [130]:
print(subscrptions_df['plan_name'].unique())
print(subscrptions_df['plan_type'].unique())
print(subscrptions_df['status'].unique())
print(subscrptions_df['cancel_reason'].unique())

['Enterprise' 'Basic' 'Pro']
['montly' 'monthly' 'yearly']
['canceled' 'trialing' 'CANCELED' 'Active' 'active']
['competitor' nan 'price' 'churned']


In [131]:
# Converting all date strings to datetime standard format
date_columns = ['trial_start_date', 'trial_end_date', 'billing_cycle_end_date','billing_cycle_start_date']
subscrptions_df[date_columns] = subscrptions_df[date_columns].apply(pd.to_datetime, format='mixed', errors='coerce')

# Clean plan_type column
subscrptions_df['plan_type'] = subscrptions_df['plan_type'].replace('montly','monthly')

# Clean status column
subscrptions_df['status'] = subscrptions_df['status'].str.title()

# as is visible from the describe(), account_id has multiple entries, we'll take the latest billing_cycle_end date
# for each account_id as it's current status in system and drop the rest, before that, we'll keep a copy
subscrptions_df_copy = subscrptions_df.copy()

subscrptions_df = (
    subscrptions_df.sort_values(by=["account_id", "billing_cycle_end_date"], ascending=[True, False])
      .drop_duplicates(subset=["account_id"], keep="first")
      .reset_index(drop=True)
)

# Dropping the is_trail field as its conflicting with status column
subscrptions_df.drop(columns=['is_trial'], inplace=True)

subscrptions_df.head(10)

,subscription_id,account_id,plan_name,plan_type,billing_cycle_start_date,billing_cycle_end_date,status,trial_start_date,trial_end_date,conversion_flag,renewal_count,cancel_reason
0,SUB2480,ACC1000,Enterprise,monthly,2025-07-21,2025-07-22,Active,2025-07-12,2025-08-04,yes,0.0,price
1,SUB4336,ACC1003,Enterprise,monthly,2025-08-02,2025-06-30,Trialing,2025-08-03,2025-06-19,yes,NaN,churned
2,SUB2839,ACC1004,Pro,yearly,2025-08-22,2025-07-17,Canceled,2025-08-03,2025-06-15,yes,-1.0,price
3,SUB3189,ACC1006,Enterprise,monthly,2025-06-01,2025-08-24,Canceled,2025-07-20,2025-08-18,yes,NaN,price
4,SUB2816,ACC1008,Pro,yearly,2025-08-04,2025-08-02,Canceled,2025-08-07,2025-07-27,yes,-1.0,price
5,SUB3814,ACC1009,Pro,monthly,2025-08-23,2025-08-30,Active,2025-06-08,2025-06-04,yes,-1.0,churned
6,SUB3586,ACC1010,Enterprise,monthly,2025-07-21,2025-07-03,Canceled,2025-06-05,2025-08-18,yes,NaN,churned
7,SUB2604,ACC1013,Basic,yearly,2025-08-22,2025-06-16,Active,2025-07-20,2025-06-11,yes,0.0,competitor
8,SUB3645,ACC1015,Enterprise,yearly,2025-07-24,2025-07-26,Canceled,2025-08-15,2025-07-27,no,2.0,price
9,SUB2228,ACC1020,Pro,yearly,2025-06-27,2025-08-14,Canceled,2025-08-05,2025-08-28,no,NaN,competitor


In [132]:
subscrptions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977 entries, 0 to 1976
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   subscription_id           1977 non-null   object        
 1   account_id                1977 non-null   object        
 2   plan_name                 1977 non-null   object        
 3   plan_type                 1977 non-null   object        
 4   billing_cycle_start_date  1977 non-null   datetime64[ns]
 5   billing_cycle_end_date    1977 non-null   datetime64[ns]
 6   status                    1977 non-null   object        
 7   trial_start_date          1977 non-null   datetime64[ns]
 8   trial_end_date            1977 non-null   datetime64[ns]
 9   conversion_flag           1977 non-null   object        
 10  renewal_count             1313 non-null   float64       
 11  cancel_reason             1502 non-null   object        
dtypes: datetime64[ns](4)

In [133]:
subscrptions_df.describe(include='all')

,subscription_id,account_id,plan_name,plan_type,billing_cycle_start_date,billing_cycle_end_date,status,trial_start_date,trial_end_date,conversion_flag,renewal_count,cancel_reason
count,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,1313.000000,1502
unique,1977,1977,3,2,NaN,NaN,3,NaN,NaN,2,NaN,3
top,SUB2480,ACC1000,Pro,monthly,NaN,NaN,Canceled,NaN,NaN,no,NaN,churned
freq,1,1,673,1312,NaN,NaN,789,NaN,NaN,1008,NaN,533
mean,NaN,NaN,NaN,NaN,2025-07-16 23:53:26.676783104,2025-07-19 08:21:51.077390080,NaN,2025-07-16 03:30:30.045523712,2025-07-15 06:27:29.772382464,NaN,0.483625,NaN
min,NaN,NaN,NaN,NaN,2025-06-01 00:00:00,2025-06-01 00:00:00,NaN,2025-06-01 00:00:00,2025-06-01 00:00:00,NaN,-1.000000,NaN
25%,NaN,NaN,NaN,NaN,2025-06-24 00:00:00,2025-06-27 00:00:00,NaN,2025-06-25 00:00:00,2025-06-23 00:00:00,NaN,-1.000000,NaN
50%,NaN,NaN,NaN,NaN,2025-07-18 00:00:00,2025-07-20 00:00:00,NaN,2025-07-16 00:00:00,2025-07-15 00:00:00,NaN,0.000000,NaN
75%,NaN,NaN,NaN,NaN,2025-08-08 00:00:00,2025-08-12 00:00:00,NaN,2025-08-08 00:00:00,2025-08-06 00:00:00,NaN,1.000000,NaN
max,NaN,NaN,NaN,NaN,2025-08-30 00:00:00,2025-08-30 00:00:00,NaN,2025-08-30 00:00:00,2025-08-30 00:00:00,NaN,2.000000,NaN


##### Dataset

In [134]:
subscrptions_df.sample(10)

,subscription_id,account_id,plan_name,plan_type,billing_cycle_start_date,billing_cycle_end_date,status,trial_start_date,trial_end_date,conversion_flag,renewal_count,cancel_reason
1447,SUB4149,ACC4709,Enterprise,yearly,2025-06-26,2025-08-19,Canceled,2025-07-19,2025-06-14,yes,1.0,competitor
98,SUB3764,ACC1228,Enterprise,yearly,2025-07-18,2025-07-30,Active,2025-08-16,2025-07-22,no,2.0,churned
1677,SUB2226,ACC5326,Pro,yearly,2025-08-01,2025-07-25,Trialing,2025-07-03,2025-06-11,yes,NaN,NaN
53,SUB3920,ACC1141,Pro,monthly,2025-07-18,2025-08-07,Canceled,2025-08-18,2025-08-08,yes,-1.0,competitor
34,SUB3812,ACC1094,Pro,monthly,2025-06-15,2025-07-20,Active,2025-06-27,2025-07-03,yes,NaN,price
698,SUB3924,ACC2807,Enterprise,monthly,2025-06-24,2025-07-30,Canceled,2025-08-24,2025-07-16,yes,-1.0,NaN
227,SUB3853,ACC1556,Enterprise,monthly,2025-06-24,2025-08-26,Active,2025-08-20,2025-07-05,yes,0.0,NaN
984,SUB3288,ACC3578,Basic,monthly,2025-06-30,2025-08-04,Active,2025-06-20,2025-06-07,no,-1.0,competitor
1341,SUB2243,ACC4433,Basic,monthly,2025-08-26,2025-08-18,Active,2025-06-06,2025-06-20,no,1.0,churned
258,SUB2589,ACC1633,Enterprise,yearly,2025-07-27,2025-07-08,Active,2025-06-25,2025-08-03,yes,NaN,price


#### Inspect

In [87]:
status_counts = subscrptions_df['status'].value_counts()
status_counts

status
Canceled    789
Active      786
Trialing    402
Name: count, dtype: int64

In [88]:
plan_wise_status = subscrptions_df.pivot_table(
    index="plan_name",        # rows
    columns="status",     # columns
    values="account_id",      # what to count
    aggfunc="count",     # how to aggregate
    fill_value=0         # replace NaN with 0
)

plan_wise_status

status,Active,Canceled,Trialing
plan_name,,,
Basic,256,253,146
Enterprise,259,257,133
Pro,271,279,123


In [93]:
#Cancelled reason is consideredo nly for status = cancelled 
cancel_reasons = subscrptions_df[subscrptions_df['status'] == 'Canceled'].cancel_reason.value_counts()

cancel_reasons

cancel_reason
churned       219
price         191
competitor    191
Name: count, dtype: int64

### Dataset preparation for Funnel & Attribution Analysis 

##### We'll design a funnel that looks like signups > trials subscription > subscriptions > dropoffs

In [117]:
# join accounts dataset with subscriptions dataset
accounts_subscriptions_df = account_df.merge(subscrptions_df[['account_id','status','renewal_count','conversion_flag','billing_cycle_end_date','trial_end_date','cancel_reason']], on='account_id', how='left')
accounts_subscriptions_df.sample(10)

,account_id,signup_date,utm_source,utm_campaign,ref,account_type,month_year,status,renewal_count,conversion_flag,billing_cycle_end_date,trial_end_date,cancel_reason
1136,ACC2136,2025-07-15,LinkedIn,Campaign_B,NaN,Agency,2025-07,Active,-1.0,no,2025-07-27,2025-08-29,churned
3407,ACC4407,2025-08-01,Google Ads,Campaign_C,test,Normal,2025-08,NaN,NaN,NaN,NaT,NaT,NaN
3249,ACC4249,2025-08-01,Organic,Campaign_B,huge,Agency,2025-08,NaN,NaN,NaN,NaT,NaT,NaN
3611,ACC4611,2025-08-05,Google Ads,Campaign_B,NaN,Normal,2025-08,Active,2.0,no,2025-06-08,2025-06-17,NaN
1263,ACC2263,2025-07-15,Referral,Campaign_B,NaN,Agency,2025-07,NaN,NaN,NaN,NaT,NaT,NaN
3156,ACC4156,2025-07-15,Organic,Campaign_A,change,Agency,2025-07,NaN,NaN,NaN,NaT,NaT,NaN
1859,ACC2859,2025-08-16,Referral,Campaign_B,reflect,Normal,2025-08,NaN,NaN,NaN,NaT,NaT,NaN
156,ACC1156,2025-07-15,Google Ads,Campaign_A,NaN,Agency,2025-07,Active,2.0,yes,2025-06-03,2025-07-06,competitor
2983,ACC3983,2025-07-18,Google Ads,Campaign_B,NaN,Normal,2025-07,NaN,NaN,NaN,NaT,NaT,NaN
3155,ACC4155,2025-08-08,Google Ads,Campaign_A,NaN,Normal,2025-08,Active,2.0,yes,2025-06-27,2025-07-18,price


In [118]:
accounts_subscriptions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   account_id              5000 non-null   object        
 1   signup_date             5000 non-null   datetime64[ns]
 2   utm_source              5000 non-null   object        
 3   utm_campaign            5000 non-null   object        
 4   ref                     1717 non-null   object        
 5   account_type            5000 non-null   object        
 6   month_year              5000 non-null   period[M]     
 7   status                  1955 non-null   object        
 8   renewal_count           1298 non-null   float64       
 9   conversion_flag         1955 non-null   object        
 10  billing_cycle_end_date  1955 non-null   datetime64[ns]
 11  trial_end_date          1955 non-null   datetime64[ns]
 12  cancel_reason           1486 non-null   object  

In [ ]:
#Assuming all accounts that have a null status as not subscribed
accounts_subscriptions_df['status'] = accounts_subscriptions_df['status'].fillna('Not subscibed')

# Add status_new column
def get_new_status(row):
    if row['status'] == 'Canceled' and row['conversion_flag'] == 'yes':
        return 'churned'
    if row['status'] == 'Canceled' and row['conversion_flag'] == 'no':
        return 'dropped after trial'
    else:
        return row['status']
    
accounts_subscriptions_df['status_new'] = accounts_subscriptions_df.apply(get_new_status, axis=1)
accounts_subscriptions_df.sample(10)

,account_id,signup_date,utm_source,utm_campaign,ref,account_type,month_year,status,renewal_count,conversion_flag,billing_cycle_end_date,trial_end_date,cancel_reason,status_new
2774,ACC3774,2025-07-15,Google Ads,Campaign_C,NaN,Normal,2025-07,Trialing,NaN,no,2025-06-14,2025-06-11,competitor,Trialing
1124,ACC2124,2025-08-01,Google Ads,Campaign_C,NaN,Agency,2025-08,Not subscibed,NaN,NaN,NaT,NaT,NaN,Not subscibed
108,ACC1108,2025-06-10,Referral,Campaign_B,NaN,Agency,2025-06,Not subscibed,NaN,NaN,NaT,NaT,NaN,Not subscibed
2067,ACC3067,2025-08-01,Unknown,Campaign_B,NaN,Normal,2025-08,Not subscibed,NaN,NaN,NaT,NaT,NaN,Not subscibed
4902,ACC5902,2025-06-25,Google Ads,Campaign_C,NaN,Normal,2025-06,Canceled,NaN,no,2025-07-21,2025-08-20,NaN,dropped after trial
1015,ACC2015,2025-07-15,LinkedIn,Campaign_C,available,Normal,2025-07,Not subscibed,NaN,NaN,NaT,NaT,NaN,Not subscibed
2397,ACC3397,2025-07-15,Referral,Campaign_A,NaN,Agency,2025-07,Not subscibed,NaN,NaN,NaT,NaT,NaN,Not subscibed
4842,ACC5842,2025-07-21,LinkedIn,Campaign_C,NaN,Normal,2025-07,Canceled,1.0,no,2025-07-21,2025-07-20,competitor,dropped after trial
3712,ACC4712,2025-08-11,Google Ads,Campaign_A,war,Normal,2025-08,Not subscibed,NaN,NaN,NaT,NaT,NaN,Not subscibed
3457,ACC4457,2025-06-11,Referral,Campaign_B,NaN,Normal,2025-06,Active,0.0,no,2025-06-22,2025-06-26,churned,Active


In [126]:
accounts_subscriptions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   account_id              5000 non-null   object        
 1   signup_date             5000 non-null   datetime64[ns]
 2   utm_source              5000 non-null   object        
 3   utm_campaign            5000 non-null   object        
 4   ref                     1717 non-null   object        
 5   account_type            5000 non-null   object        
 6   month_year              5000 non-null   period[M]     
 7   status                  5000 non-null   object        
 8   renewal_count           1298 non-null   float64       
 9   conversion_flag         1955 non-null   object        
 10  billing_cycle_end_date  1955 non-null   datetime64[ns]
 11  trial_end_date          1955 non-null   datetime64[ns]
 12  cancel_reason           1486 non-null   object  

In [125]:
accounts_subscriptions_df.to_csv('../data/accounts_subscriptions_df.csv',index=False)

### Dataset preparation for Cohort & Retention Analysis 
* We'll calculate retention from activities of accounts in server as that provides most number of datapoints, 
* We'll use utm_source and utm_campaigns to group customers as these values are consistent and don't change. This will be taken from accounts dataset
* Ideally the event_date should be greater than signup_date, but its not always the case here, So just to have more data points, we'll keep the event date and ignore signup_date

In [293]:
retention_df = server_events_df[server_events_df['account_id']!=''].merge(accounts_subscriptions_df[['account_id','utm_source','utm_campaign']], on='account_id', how='left')
retention_df.drop(columns=['month_year'], inplace=True)
retention_df.head()

,date,account_id,event,page,type,feature,utm_source,utm_campaign
0,2025-07-15,ACC5981,error,NaN,load_error,NaN,LinkedIn,Campaign_B
1,2025-07-15,ACC2061,feature_use,NaN,NaN,cookie_banner,Unknown,Campaign_A
2,2025-07-12,ACC3989,error,NaN,load_error,NaN,Unknown,Campaign_B
3,2025-07-15,ACC3902,error,NaN,unknown,NaN,Google Ads,Campaign_A
4,2025-08-29,ACC5315,error,NaN,load_error,NaN,Referral,Campaign_B


In [294]:
retention_df['first_activity_date'] = retention_df.groupby('account_id')['date'].transform('min')

def m1_retention(row):
    days_diff = (row['date'] - row['first_activity_date']).days
    if  days_diff >0 and days_diff < 30:
        return row['account_id']
    else:
        return None  
    
def m2_retention(row):
    days_diff = (row['date'] - row['first_activity_date']).days
    if  days_diff >= 30 and days_diff < 60:
        return row['account_id']
    else:
        return None  
    
def m3_retention(row):
    days_diff = (row['date'] - row['first_activity_date']).days
    if  days_diff >= 60 and days_diff < 90:
        return row['account_id']
    else:
        return None 

# Apply the function to create a retention columns
retention_df['m1'] = retention_df.apply(m1_retention, axis=1)
retention_df['m2'] = retention_df.apply(m2_retention, axis=1)
retention_df['m3'] = retention_df.apply(m3_retention, axis=1)

# Extract month_year from first_activity_date
retention_df['activity_month'] = retention_df['first_activity_date'].dt.to_period('M')

retention_df.sample(10)
retention_df.head()

,date,account_id,event,page,type,feature,utm_source,utm_campaign,first_activity_date,m1,m2,m3,activity_month
0,2025-07-15,ACC5981,error,NaN,load_error,NaN,LinkedIn,Campaign_B,2025-07-15,None,None,None,2025-07
1,2025-07-15,ACC2061,feature_use,NaN,NaN,cookie_banner,Unknown,Campaign_A,2025-07-15,None,None,None,2025-07
2,2025-07-12,ACC3989,error,NaN,load_error,NaN,Unknown,Campaign_B,2025-06-10,None,ACC3989,None,2025-06
3,2025-07-15,ACC3902,error,NaN,unknown,NaN,Google Ads,Campaign_A,2025-07-15,None,None,None,2025-07
4,2025-08-29,ACC5315,error,NaN,load_error,NaN,Referral,Campaign_B,2025-07-05,None,ACC5315,None,2025-07


In [295]:
all_accounts = retention_df['account_id'].nunique()
print('All accounts :',all_accounts,' 100%')

m1 = retention_df['m1'].nunique()
print('m1           :',m1, ' ',round((m1/all_accounts)*100,1),'%')

m2 = retention_df['m2'].nunique()
print('m2           :',m2,' ',round((m2/all_accounts)*100,1),'%')

m3 = retention_df['m3'].nunique()
print('m3           :',m3,'  ',round((m3/all_accounts)*100,1),'%')

All accounts : 3197  100%
m1           : 681   21.3 %
m2           : 436   13.6 %
m3           : 66    2.1 %


In [ ]:
m1_retention_campaign_wise  = retention_df.groupby(["activity_month", "utm_campaign"]).agg(
    m1 = ("m1", "nunique"),
    accounts = ('account_id','nunique')
).reset_index()


m1_retention_campaign_wise['m1 %']= round((m1_retention_campaign_wise['m1']/m1_retention_campaign_wise['accounts'])*100,2)
m1_retention_campaign_wise = m1_retention_campaign_wise.pivot_table(index='activity_month', columns='utm_campaign', values='m1 %', fill_value=0).reset_index()
m1_retention_campaign_wise.to_csv('../data/m1_retention_campaign_wise.csv',index=False)
m1_retention_campaign_wise

utm_campaign,activity_month,Campaign_A,Campaign_B,Campaign_C
0,2025-06,27.55,32.63,30.60
1,2025-07,20.72,19.15,21.78
2,2025-08,6.25,6.36,8.79


In [340]:
m2_retention_campaign_wise  = retention_df.groupby(["activity_month", "utm_campaign"]).agg(
    m2 = ("m2", "nunique"),
    accounts = ('account_id','nunique')
).reset_index()

# m1_retention_source_wise.drop(columns=['utm_campaign'], inplace=True)
# m1_retention_source_wise['Campaign_A'] = round((m1_retention_source_wise['Campaign_A']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_A'].nunique())*100,2)
# m1_retention_source_wise['Campaign_B'] = round((m1_retention_source_wise['Campaign_B']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_B'].nunique())*100,2)
# m1_retention_source_wise['Campaign_C'] = round((m1_retention_source_wise['Campaign_C']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_C'].nunique())*100,2)
m2_retention_campaign_wise['m2 %']= round((m2_retention_campaign_wise['m2']/m2_retention_campaign_wise['accounts'])*100,2)
m2_retention_campaign_wise = m2_retention_campaign_wise.pivot_table(index='activity_month', columns='utm_campaign', values='m2 %', fill_value=0).reset_index()
m2_retention_campaign_wise.to_csv('../data/m2_retention_campaign_wise.csv',index=False)
m2_retention_campaign_wise

utm_campaign,activity_month,Campaign_A,Campaign_B,Campaign_C
0,2025-06,37.76,29.47,38.25
1,2025-07,8.66,8.64,7.88
2,2025-08,0.00,0.00,0.00


In [341]:
m3_retention_campaign_wise  = retention_df.groupby(["activity_month", "utm_campaign"]).agg(
    m3 = ("m3", "nunique"),
    accounts = ('account_id','nunique')
).reset_index()

# m1_retention_source_wise.drop(columns=['utm_campaign'], inplace=True)
# m1_retention_source_wise['Campaign_A'] = round((m1_retention_source_wise['Campaign_A']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_A'].nunique())*100,2)
# m1_retention_source_wise['Campaign_B'] = round((m1_retention_source_wise['Campaign_B']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_B'].nunique())*100,2)
# m1_retention_source_wise['Campaign_C'] = round((m1_retention_source_wise['Campaign_C']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_C'].nunique())*100,2)
m3_retention_campaign_wise['m3 %']= round((m3_retention_campaign_wise['m3']/m3_retention_campaign_wise['accounts'])*100,2)
m3_retention_campaign_wise = m3_retention_campaign_wise.pivot_table(index='activity_month', columns='utm_campaign', values='m3 %', fill_value=0).reset_index()
m3_retention_campaign_wise.to_csv('../data/m3_retention_campaign_wise.csv',index=False)
m3_retention_campaign_wise

utm_campaign,activity_month,Campaign_A,Campaign_B,Campaign_C
0,2025-06,6.63,7.89,12.02
1,2025-07,0.00,0.00,0.00
2,2025-08,0.00,0.00,0.00


In [342]:
m1_retention_source_wise  = retention_df.groupby(["activity_month", "utm_source"]).agg(
    m1 = ("m1", "nunique"),
    accounts = ('account_id','nunique')
).reset_index()

# m1_retention_source_wise.drop(columns=['utm_campaign'], inplace=True)
# m1_retention_source_wise['Campaign_A'] = round((m1_retention_source_wise['Campaign_A']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_A'].nunique())*100,2)
# m1_retention_source_wise['Campaign_B'] = round((m1_retention_source_wise['Campaign_B']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_B'].nunique())*100,2)
# m1_retention_source_wise['Campaign_C'] = round((m1_retention_source_wise['Campaign_C']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_C'].nunique())*100,2)
m1_retention_source_wise['m1 %']= round((m1_retention_source_wise['m1']/m1_retention_source_wise['accounts'])*100,2)
m1_retention_source_wise = m1_retention_source_wise.pivot_table(index='activity_month', columns='utm_source', values='m1 %', fill_value=0).reset_index()
m1_retention_source_wise.to_csv('../data/m1_retention_source_wise.csv',index=False)
m1_retention_source_wise

utm_source,activity_month,Google Ads,LinkedIn,Organic,Referral,Unknown
0,2025-06,32.28,31.69,33.06,31.40,23.73
1,2025-07,21.01,19.38,18.13,18.79,22.82
2,2025-08,2.88,4.17,9.84,10.71,10.17


In [343]:
m2_retention_source_wise  = retention_df.groupby(["activity_month", "utm_source"]).agg(
    m2 = ("m2", "nunique"),
    accounts = ('account_id','nunique')
).reset_index()

# m1_retention_source_wise.drop(columns=['utm_campaign'], inplace=True)
# m1_retention_source_wise['Campaign_A'] = round((m1_retention_source_wise['Campaign_A']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_A'].nunique())*100,2)
# m1_retention_source_wise['Campaign_B'] = round((m1_retention_source_wise['Campaign_B']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_B'].nunique())*100,2)
# m1_retention_source_wise['Campaign_C'] = round((m1_retention_source_wise['Campaign_C']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_C'].nunique())*100,2)
m2_retention_source_wise['m2 %']= round((m2_retention_source_wise['m2']/m2_retention_source_wise['accounts'])*100,2)
m2_retention_source_wise = m2_retention_source_wise.pivot_table(index='activity_month', columns='utm_source', values='m2 %', fill_value=0).reset_index()
m2_retention_source_wise.to_csv('../data/m2_retention_source_wise.csv',index=False)
m2_retention_source_wise

utm_source,activity_month,Google Ads,LinkedIn,Organic,Referral,Unknown
0,2025-06,35.04,34.51,29.03,28.1,40.68
1,2025-07,9.28,6.46,7.93,7.8,10.21
2,2025-08,0.00,0.00,0.00,0.0,0.00


In [344]:
m3_retention_source_wise  = retention_df.groupby(["activity_month", "utm_source"]).agg(
    m3 = ("m3", "nunique"),
    accounts = ('account_id','nunique')
).reset_index()

# m1_retention_source_wise.drop(columns=['utm_campaign'], inplace=True)
# m1_retention_source_wise['Campaign_A'] = round((m1_retention_source_wise['Campaign_A']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_A'].nunique())*100,2)
# m1_retention_source_wise['Campaign_B'] = round((m1_retention_source_wise['Campaign_B']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_B'].nunique())*100,2)
# m1_retention_source_wise['Campaign_C'] = round((m1_retention_source_wise['Campaign_C']/retention_df['account_id'][retention_df['utm_campaign']=='Campaign_C'].nunique())*100,2)
m3_retention_source_wise['m3 %']= round((m3_retention_source_wise['m3']/m3_retention_source_wise['accounts'])*100,2)
m3_retention_source_wise = m3_retention_source_wise.pivot_table(index='activity_month', columns='utm_source', values='m3 %', fill_value=0).reset_index()
m3_retention_source_wise.to_csv('../data/m3_retention_source_wise.csv',index=False)
m3_retention_source_wise

utm_source,activity_month,Google Ads,LinkedIn,Organic,Referral,Unknown
0,2025-06,7.09,7.75,8.87,14.05,6.78
1,2025-07,0.00,0.00,0.00,0.00,0.00
2,2025-08,0.00,0.00,0.00,0.00,0.00


### Dataset preparation for MRR abd ARPU Analysis

In [369]:
# We'll  take the invoices with paid status and value > 0
revenue_df = invoices_df[(invoices_df['amount_in_inr'] > 0) & ( invoices_df['status'] == 'paid')].reset_index(drop=True)
revenue_df.head()

,invoice_id,subscription_id,account_id,invoice_date,amount,currency,status,amount_in_inr
0,INV5000,SUB3890,ACC1638,2025-09-15,99.43,EUR,paid,8948.70
1,INV5002,SUB3219,ACC3121,2025-09-15,472.47,USD,paid,39215.01
2,INV5003,SUB3706,ACC2800,2025-08-29,160.58,USD,paid,13328.14
3,INV5012,SUB4418,ACC1066,2025-09-15,140.97,INR,paid,140.97
4,INV5016,SUB4333,ACC1918,2025-08-22,226.22,USD,paid,18776.26


In [371]:
revenue_df['invoice_date_year_month'] = revenue_df['invoice_date'].dt.to_period('M')
region = {
    'INR': 'India',
    'EUR': 'Europe',
    'USD': 'USA'
}
revenue_df['region'] = revenue_df['currency'].map(region)
revenue_df.head()

,invoice_id,subscription_id,account_id,invoice_date,amount,currency,status,amount_in_inr,invoice_date_year_month,region
0,INV5000,SUB3890,ACC1638,2025-09-15,99.43,EUR,paid,8948.70,2025-09,Europe
1,INV5002,SUB3219,ACC3121,2025-09-15,472.47,USD,paid,39215.01,2025-09,USA
2,INV5003,SUB3706,ACC2800,2025-08-29,160.58,USD,paid,13328.14,2025-08,USA
3,INV5012,SUB4418,ACC1066,2025-09-15,140.97,INR,paid,140.97,2025-09,India
4,INV5016,SUB4333,ACC1918,2025-08-22,226.22,USD,paid,18776.26,2025-08,USA


In [375]:
mrr_and_arpu = revenue_df.groupby(['invoice_date_year_month']).agg(
    MRR = ('amount_in_inr','sum'),
    customers = ('account_id','nunique')
    ).reset_index()

mrr_and_arpu['ARPU'] = round(mrr_and_arpu['MRR']/mrr_and_arpu['customers'],2)
mrr_and_arpu.to_csv('../data/mrr_and_arpu.csv',index=False)
mrr_and_arpu.head(10)

,invoice_date_year_month,MRR,customers,ARPU
0,2025-06,1393491.01,77,18097.29
1,2025-07,929801.88,63,14758.76
2,2025-08,1264924.13,64,19764.44
3,2025-09,3628247.66,220,16492.03
